# HuBMAP masks and probability density function

> Idea: Knowing that the glomeruli are mainly found in the cortex, we should focus on this region during training.

## Background

A glomerulus is a network of small blood vessels located at the beginning of a nephron in the kidney ([Wikipedia](https://en.wikipedia.org/wiki/Glomerulus_(kidney))). Glomeruli are mainly found in the renal **cortex**, while the renal **medulla** contains mainly the renal tubule. Since we are dealing with biological structures, the separation is not not absolute and the transitions are not always perfectly sharp.

![Diagram of a nephron](http://s3-us-west-2.amazonaws.com/courses-images/wp-content/uploads/sites/1842/2017/05/26234530/m9skcbftjqzrokkkopam.png)
[Diagram of a nephron from libretexts.org, Introductory and General Biology](https://bio.libretexts.org/Bookshelves/Introductory_and_General_Biology/Book%3A_General_Biology_(Boundless)/41%3A_Osmotic_Regulation_and_the_Excretory_System/41.4%3A_Human_Osmoregulatory_and_Excretory_Systems/41.4B%3A_Nephron%3A_The_Functional_Unit_of_the_Kidney)

## Key Idea

A common approach to deal with the very large (>500MB - 5GB) TIFF files in the dataset is to decompose the images in smaller patches/tiles, for instance by using a sliding window apporach.
> **Knowing that the glomeruli are mainly found in the cortex, we should focus on this region during training**. 

Instead of preprocessing the images by saving them into fixed tiles, we prepare the data to sample random tiles from the entire images with a higher probability to sample tiles that contain glomeruli and cortex.

**We combine two sampling apporaches**

> 1. Sampling tiles that contain all glomeruli - this ensures that each glomerulus is seen during one epoch of training at least once. 
> 2. Sampling random tiles based on region probabilites (e.g., medulla; cortex)

*Note: We startet out by only using the second approach (see our public kernel https://www.kaggle.com/matjes/hubmap-labels-pdf-0-5-0-25-0-01). However, we realized that due to the randomness of the sampling sometimes not all glomeruli were drawn during training. Thus, we added the first step.*

## Advantages of this approaches

In combination with [deepflash2](https://github.com/matjesg/deepflash2/tree/master/) and the deepflash2 [pytorch datasets](https://matjesg.github.io/deepflash2/data.html#Datasets) in particular, this approach has several advantages:
- no preprocessing of the data (only saving them to .zarr files for memory efficient loading)
    - flexible tile shapes (input shapes, e.g. 1024, 512, 256) at runtime
    - flexible scaling (e.g., by facors of 2,3,4)
- faster convergence during traing
    - focusing on the relevant regions (e.g., tiles that contain glomeruli and cortex)
    - "additional" data augmentation from random sampling (compared to fixed windows)

## Implementation (masks and probability density function)

The data preprocessing steps in this notebook comprise
- Creating segmentations masks from RLE
- Creating a data frame containing position information on each glomerulus in the segmentation mask
- Creating probability density function (PDF) for efficient sampling from mask and anatmical structure

**Inputs**
- RLE glomeruli segmentations ([challenge dataset](https://www.kaggle.com/c/hubmap-kidney-segmentation/data))
- Anatomical region masks ([.zarr converted](https://www.kaggle.com/matjes/hubmap-pdf-zarr))
   
**Settings**

- Probability for cortex regions (`cortex_p`): 0.7 
- Probability for medulla regions (`medulla_p`): 0.2
- Probability for other regions (`bg_p` (background)): 0.1    

We chose the region sampling probabilites based on expert knowledge and some experiements. However, these settings are hyperparamteres that can be tuned or adjusted if the unterlying data changes.


In [ ]:
# Install zarr and load packages
!pip install -qq zarr
import cv2, zarr, gc
import matplotlib.pyplot as plt, numpy as np, pandas as pd
from pathlib import Path
gc.enable()

In [ ]:
# from https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [
        np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape).T

Settings

In [ ]:
class CONFIG:
    path = Path('/kaggle/input/hubmap-kidney-segmentation')
    anatomy = '../input/hubmap-anatomy-zarr/anatomy'
    
    scale = 2        # Downscale final mask by factor 2
    cdf_size = 512   # Downscale CDF for memory efficient loading during training
    bg_p = 0.1       # Background Probability
    cortex_p = 0.7   # Cortex Probability
    medulla_p = 0.2  # Medulla Probability
    
cfg = CONFIG()

**Load inputs and define outputs**

In [ ]:
# Input 
df_train = pd.read_csv(cfg.path/"train.csv",index_col='id')
df_info = pd.read_csv(cfg.path/"HuBMAP-20-dataset_information.csv")
grp_pdf = zarr.open_group(cfg.anatomy) # Anatomy labels sever as basis for PDF

# Output
root = zarr.group(f'/kaggle/working/masks_scale{cfg.scale}')
# Saving cdf in 'pdfs' due to naming conventions for sampling during training in deepflash2
g_msk, g_pdf, g_cdf = root.create_groups('labels', 'pdfs', 'cdfs', overwrite=True)

**Loop over files to create...**
1. The segmentation mask (.zarr)
1. A list of glomeruli for *glomeruli sampling*
1. The probability density function for *region sampling*

In [ ]:
df_list = []
for idx, row in df_train.iterrows():
    
    # Get image info
    img_info = df_info[df_info.image_file==f'{idx}.tiff']
    shape = (img_info.height_pixels.values[0], img_info.width_pixels.values[0])
    print(idx, 'with shape', shape)
    
    msk = rle2mask(row.encoding, (shape[1], shape[0])).astype('uint8')
    
    # Plot
    fig, ax = plt.subplots(ncols=2, figsize=(15,15))
    resize_w = int((msk.shape[1]/msk.shape[0])*cfg.cdf_size)
    ax[0].imshow(cv2.resize(msk, dsize=(resize_w, cfg.cdf_size)))
    ax[0].set_title('Mask')
    ax[0].set_axis_off()
    
    pdf = grp_pdf[idx][:]
    
    if cfg.scale!=1:
        new_size = (msk.shape[1] // cfg.scale, msk.shape[0] // cfg.scale)
        print('Scaling to', new_size)
        msk = cv2.resize(msk, new_size)
        pdf = cv2.resize(pdf, new_size)
        
    pdf = pdf.astype('float32')          
    pdf[pdf==0] = cfg.bg_p/np.sum(pdf==0)
    pdf[msk>0] = 0
    pdf[pdf==1] = cfg.cortex_p/np.sum(pdf==1)
    pdf[pdf==2] = cfg.medulla_p/np.sum(pdf==2)    
      
    print('Getting glomeruli stats')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(msk, connectivity=4)
    print(f'Found {nb_components} glomeruli')
    df_centroids = pd.DataFrame(centroids[1:], columns=['cy', 'cx'])
    df_centroids = df_centroids.join(pd.DataFrame(stats[1:], columns=['left', 'top', 'width', 'height', 'area']))
    df_centroids['idx'] = idx 
    df_centroids.reset_index(inplace=True)
    df_centroids.set_index(['idx', 'index'], inplace=True)
    df_list.append(df_centroids)
    
    # Saving 
    g_msk[idx] = msk
    g_pdf[idx] = pdf
    
    # Saving cdf
    pdf = cv2.resize(pdf, dsize=(resize_w, cfg.cdf_size))      
    g_cdf[idx] = np.cumsum(pdf/np.sum(pdf)) 
            
    ax[1].imshow(pdf)
    ax[1].set_title('Probability density function for sampling')
    ax[1].set_axis_off() 
    plt.show()


df_stats = pd.concat(df_list)
df_stats.to_csv(f'/kaggle/working/masks_scale{cfg.scale}/roi_stats.csv')
df_stats